In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import csv
import os

COLOR_FONDO = "#e6ffe6"  # Fondo verde menta

# ---------- Funciones de datos ----------
def inicializar_csv():
    if not os.path.exists("finanzas.csv"):
        with open("finanzas.csv", "w", newline="", encoding="utf-8-sig") as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(["Fecha", "Descripción", "Monto", "Tipo"])

def agregar_registro(fecha, descripcion, monto, tipo):
    with open("finanzas.csv", "a", newline="", encoding="utf-8-sig") as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([fecha, descripcion, monto, tipo])

def cargar_registros():
    registros = []
    if os.path.exists("finanzas.csv"):
        with open("finanzas.csv", "r", encoding="utf-8-sig") as archivo:
            lector = csv.reader(archivo)
            next(lector, None)
            registros = list(lector)
    return registros

def guardar_todos(registros):
    with open("finanzas.csv", "w", newline="", encoding="utf-8-sig") as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow(["Fecha", "Descripción", "Monto", "Tipo"])
        escritor.writerows(registros)

# ---------- Ventana principal ----------
def abrir_principal(usuario):
    ventana = tk.Tk()
    ventana.title("Gestor de Finanzas")
    ventana.geometry("800x500")
    ventana.configure(bg=COLOR_FONDO)

    tk.Label(ventana, text=f"Bienvenido, {usuario}", font=("Arial", 16, "bold"), bg=COLOR_FONDO, fg="#006400").pack(pady=10)
    tk.Label(ventana, text="Registra, edita o elimina tus ingresos y gastos", font=("Arial", 12), bg=COLOR_FONDO).pack()

    frame_formulario = tk.Frame(ventana, bg=COLOR_FONDO)
    frame_formulario.pack(pady=10)

    tk.Label(frame_formulario, text="Fecha:", bg=COLOR_FONDO).grid(row=0, column=0, padx=5, pady=5)
    entry_fecha = tk.Entry(frame_formulario)
    entry_fecha.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(frame_formulario, text="Descripción:", bg=COLOR_FONDO).grid(row=1, column=0, padx=5, pady=5)
    entry_descripcion = tk.Entry(frame_formulario)
    entry_descripcion.grid(row=1, column=1, padx=5, pady=5)

    tk.Label(frame_formulario, text="Monto:", bg=COLOR_FONDO).grid(row=2, column=0, padx=5, pady=5)
    entry_monto = tk.Entry(frame_formulario)
    entry_monto.grid(row=2, column=1, padx=5, pady=5)

    tk.Label(frame_formulario, text="Tipo:", bg=COLOR_FONDO).grid(row=3, column=0, padx=5, pady=5)
    combo_tipo = ttk.Combobox(frame_formulario, values=["Ingreso", "Gasto"])
    combo_tipo.grid(row=3, column=1, padx=5, pady=5)

    # ---------- Tabla ----------
    frame_tabla = tk.Frame(ventana, bg=COLOR_FONDO)
    frame_tabla.pack(pady=10)

    tabla = ttk.Treeview(frame_tabla, columns=("Fecha", "Descripción", "Monto", "Tipo"), show="headings", height=10)
    tabla.heading("Fecha", text="Fecha")
    tabla.heading("Descripción", text="Descripción")
    tabla.heading("Monto", text="Monto")
    tabla.heading("Tipo", text="Tipo")
    tabla.pack()

    def actualizar_tabla():
        for fila in tabla.get_children():
            tabla.delete(fila)
        for reg in cargar_registros():
            tabla.insert("", "end", values=reg)

    actualizar_tabla()

    # ---------- Funciones principales ----------
    def guardar_datos():
        fecha = entry_fecha.get()
        descripcion = entry_descripcion.get()
        monto = entry_monto.get()
        tipo = combo_tipo.get()
        if not (fecha and descripcion and monto and tipo):
            messagebox.showwarning("Campos incompletos", "Por favor, completa todos los campos.")
            return
        agregar_registro(fecha, descripcion, monto, tipo)
        messagebox.showinfo("Éxito", "Registro agregado correctamente.")
        limpiar_campos()
        actualizar_tabla()

    def limpiar_campos():
        entry_fecha.delete(0, tk.END)
        entry_descripcion.delete(0, tk.END)
        entry_monto.delete(0, tk.END)
        combo_tipo.set("")

    def eliminar_registro():
        seleccionado = tabla.selection()
        if not seleccionado:
            messagebox.showwarning("Aviso", "Selecciona un registro para eliminar.")
            return
        confirm = messagebox.askyesno("Confirmar", "¿Seguro que deseas eliminar este registro?")
        if confirm:
            item = tabla.item(seleccionado)
            valores = item["values"]
            registros = cargar_registros()
            registros.remove(valores)
            guardar_todos(registros)
            actualizar_tabla()
            messagebox.showinfo("Éxito", "Registro eliminado correctamente.")

    def editar_registro():
        seleccionado = tabla.selection()
        if not seleccionado:
            messagebox.showwarning("Aviso", "Selecciona un registro para editar.")
            return
        item = tabla.item(seleccionado)
        valores = item["values"]

        # Cargar los valores seleccionados en los campos
        entry_fecha.delete(0, tk.END)
        entry_fecha.insert(0, valores[0])
        entry_descripcion.delete(0, tk.END)
        entry_descripcion.insert(0, valores[1])
        entry_monto.delete(0, tk.END)
        entry_monto.insert(0, valores[2])
        combo_tipo.set(valores[3])

        # Al guardar, actualizar el registro
        def guardar_edicion():
            nueva_fecha = entry_fecha.get()
            nueva_desc = entry_descripcion.get()
            nuevo_monto = entry_monto.get()
            nuevo_tipo = combo_tipo.get()
            registros = cargar_registros()
            indice = registros.index(valores)
            registros[indice] = [nueva_fecha, nueva_desc, nuevo_monto, nuevo_tipo]
            guardar_todos(registros)
            actualizar_tabla()
            limpiar_campos()
            messagebox.showinfo("Éxito", "Registro editado correctamente.")
            btn_guardar.config(command=guardar_datos, text="Guardar")

        btn_guardar.config(command=guardar_edicion, text="Guardar Cambios")

    # ---------- Botones ----------
    frame_botones = tk.Frame(ventana, bg=COLOR_FONDO)
    frame_botones.pack(pady=10)

    btn_guardar = tk.Button(frame_botones, text="Guardar", bg="#90EE90", width=12, command=guardar_datos)
    btn_guardar.grid(row=0, column=0, padx=10)

    tk.Button(frame_botones, text="Editar", bg="#FFD700", width=12, command=editar_registro).grid(row=0, column=1, padx=10)
    tk.Button(frame_botones, text="Eliminar", bg="#FF7F7F", width=12, command=eliminar_registro).grid(row=0, column=2, padx=10)
    tk.Button(frame_botones, text="Salir", bg="#90EE90", width=12, command=ventana.destroy).grid(row=0, column=3, padx=10)

    ventana.mainloop()

# ---------- Ventana de bienvenida ----------
def iniciar_sesion():
    nombre = entry_nombre.get().strip()
    if nombre == "":
        messagebox.showwarning("Aviso", "Por favor, ingresa tu nombre.")
    else:
        messagebox.showinfo("Bienvenido", f"¡Hola, {nombre}! Bienvenido al gestor de finanzas.")
        inicio.destroy()
        inicializar_csv()
        abrir_principal(nombre)

inicio = tk.Tk()
inicio.title("Bienvenida")
inicio.geometry("500x300")
inicio.configure(bg=COLOR_FONDO)

tk.Label(inicio, text="Bienvenido al Gestor de Finanzas", font=("Arial", 18, "bold"), bg=COLOR_FONDO, fg="#006400").pack(pady=20)
tk.Label(inicio, text="Por favor, ingresa tu nombre:", font=("Arial", 12), bg=COLOR_FONDO).pack()
entry_nombre = tk.Entry(inicio, font=("Arial", 12))
entry_nombre.pack(pady=10)
tk.Button(inicio, text="Ingresar", font=("Arial", 12, "bold"), bg="#90EE90", command=iniciar_sesion).pack(pady=10)

inicio.mainloop()